In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5

In [2]:
TickDataname='GDF2022_3_27_16_52.csv'
TickGDF=pd.read_csv('CollectedData/'+TickDataname)

TGDF=TickGDF[(TickGDF['TM_GP']>=0)&(TickGDF['TM_GP']<39)]
TGDF=TGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()
new_df=new_df.reset_index()
Train_df=new_df
Train_df.loc[new_df['TM_GP']==0,'시가']=Train_df.loc[new_df['TM_GP']==1,'시가'].values

In [3]:
Train_df[['고가','저가']]=Train_df[['고가','저가']].div(Train_df['시가'],axis=0).values
Train_df[['고가','저가','직전체결가격','시가']]=np.log(Train_df[['고가','저가','직전체결가격','시가']])

In [4]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

In [5]:
qtylist=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량']

In [6]:
Train_df[qtylist]=Train_df[qtylist].div(Train_df['10단계호가합계잔량'],axis=0).values

In [7]:
Train_df['10단계호가합계잔량']=np.log(new_df['10단계호가합계잔량'])

In [8]:
ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickGDF=pd.read_csv('CollectedData/'+ORDTickDataname)

In [9]:
ORDTickGDF.columns

Index(['Unnamed: 0', 'ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO', 'ASKBID_TP_CD',
       'MODCANCL_TP_CD', 'ORD_QTY', 'ORD_ACPT_TM', 'TM_GP'],
      dtype='object')

In [10]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]

OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

In [11]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [12]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

In [13]:
feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',  '시가', '직전체결가격','NET_ORD_QTY']

In [14]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]
mbrnlist=mbrnlist1+[a for a in mbrnlist2 if a not in mbrnlist1]
mbr,brn=mbrnlist[0]

In [15]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    print(Train_df['NET_ORD_QTY'].sum())
    ylabel=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0)).values
    

    SaveDirPath='/Data/Min/TransLOB/TrainData/'
    now=datetime.now()
    TrainDataname='Train_ORD'+'_'+str(mbr)+'_'+str(brn)+str(now.year)+'_'+str(now.month)+'_'+str(now.day)
    TrainLabelDataname='Train_ORD_Label_'+'_'+str(mbr)+'_'+str(brn)+str(now.year)+'_'+str(now.month)+'_'+str(now.day)
    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,ylabel)
    with open(SaveDirPath+'DataInformation.txt', "a+") as f:
        f.write("TrainDataname:"+TrainDataname+'\n')
        f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
        f.write("TickDataname:"+TickDataname)
        f.write("ORDTickDataname:"+ORDTickDataname)
        for col in feat_cols:
            f.write(str(col)+'\t')
        f.write('\n')
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)

5 194
-33328534.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_5_1942022_4_12
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__5_1942022_4_12
2 155
1011342.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_2_1552022_4_12
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__2_1552022_4_12
12 100
-27072832.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_12_1002022_4_12
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__12_1002022_4_12
17 29
-27191045.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_17_292022_4_12
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__17_292022_4_12
42 1
2665621.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_42_12022_4_12
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__42_12022_4_12
44 1
-17303050.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_44_12022_4_12
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__44_12022_4_12
50 92
-2678352.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_50_922022_4_12
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__50_92202

a

SaveDirPath='/Data/Min/TransLOB/TrainData/'
now=datetime.now()
TrainDataname='Train_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
TrainLabelDataname='Train_Label_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
np.save(SaveDirPath+TrainLabelDataname,increment)
with open(SaveDirPath+'DataInformation.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname)
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')
print("path:",SaveDirPath+TrainDataname)
print("path:",SaveDirPath+TrainLabelDataname)